In [8]:
from pyspark.sql import SparkSession

spark_session = SparkSession\
        .builder\
        .master("spark://hadoop-master:7077") \
        .appName("DE-1-4-SparkSession")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.executor.instances", 7) \
        .config("spark.dynamicAllocation.shuffleTracking.enabled",True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max", 2) \
        .config("spark.executor.cores",2)\
        .config("spark.driver.port",9999)\
        .config("spark.blockManager.port",10005)\
        .getOrCreate()

In [7]:
spark_session.stop()

In [24]:
reddit_df = spark_session.read.json("hdfs://hadoop-master:9000/user/hadoop/input/corpus-webis-tldr-17.json")
reddit_df.printSchema()

root
 |-- author: string (nullable = true)
 |-- body: string (nullable = true)
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- id: string (nullable = true)
 |-- normalizedBody: string (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)
 |-- title: string (nullable = true)



In [9]:
path_pos = "./opinion-lexicon-English/positive-words.txt"
path_neg = "./opinion-lexicon-English/negative-words.txt"
positive_words = set()
negative_words = set()
with open(path_pos, 'r', encoding='utf-8') as file:
    for line in file:
        positive_words.add(line.strip())
with open(path_neg, 'r', encoding='utf-8') as file:
    for line in file:
        negative_words.add(line.strip())

In [26]:
reddit_df = reddit_df.drop("author", "body", "normalizedBody", "id", "subreddit_id", "title")
reddit_df.printSchema()


root
 |-- content: string (nullable = true)
 |-- content_len: long (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- summary_len: long (nullable = true)



In [29]:
#sampled_reddit_df = reddit_df.sample(False, 0.05)

In [10]:
broadcast_positive_words = spark_session.sparkContext.broadcast(positive_words)
broadcast_negative_words = spark_session.sparkContext.broadcast(negative_words)

In [11]:
import re
def pre_process(summary):
    line = summary.lower()
    line = re.sub(r"[.,]",'',line).split(" ")
    positive_count = sum([word in broadcast_positive_words.value for word in line])
    negative_count = sum([word in broadcast_negative_words.value for word in line])
    if positive_count > negative_count:
        return (1,positive_count,negative_count, 1)
    elif negative_count > positive_count:
        return (-1,positive_count,negative_count, 1)
    else:
        return (0, positive_count, negative_count, 1)
    

In [12]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType, StructType, StructField

schema = StructType([
    StructField("sentiment", IntegerType(), False),
    StructField("positive_count", IntegerType(), False),
    StructField("negative_count", IntegerType(), False),
    StructField("number_of_tweets", IntegerType(), False)
])

In [13]:
from operator import add
import time as time
start_time = time.time()
reddit_df = spark_session.read.json("hdfs://hadoop-master:9000/user/hadoop/input/corpus-webis-tldr-17.json")
reddit_df = reddit_df.drop("author", "body", "normalizedBody", "id", "subreddit_id", "title", "content", "content_len", "summary_len")
sampled_reddit_df = reddit_df.filter(reddit_df["subreddit"] != "NULL")
sampled_reddit_df = reddit_df.sample(False, 0.1)
conotation_udf = udf(pre_process, schema)
reddit_df_conotation = sampled_reddit_df.withColumn("conotation", conotation_udf("summary"))
grouped = reddit_df_conotation.rdd.map(lambda x: (x["subreddit"], (x["conotation"]["sentiment"], x["conotation"]["number_of_tweets"])))
grouped = grouped.reduceByKey(lambda a,b: (a[0] + b[0], a[1] + b[1]))
grouped.take(1)
end_time = time.time()
print(f"Total time: {end_time-start_time} seconds")

24/03/11 13:19:55 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/03/11 13:19:56 ERROR Utils: Uncaught exception in thread stop-spark-context
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.deploy.client.StandaloneAppClient.stop(StandaloneAppClient.scala:288)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.org$apache$spark$scheduler$cluster$StandaloneSchedulerBackend$$stop(StandaloneSchedulerBackend.scala:275)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.stop(StandaloneSchedulerBackend.scala:142)
	at org.apache.spark.scheduler.SchedulerBackend.stop(SchedulerBackend.scala:33)
	at org.apache.spark.scheduler.SchedulerBackend.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Master removed our application: KILLED
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1247)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler.scala:12

In [7]:
grouped = grouped.sortBy(lambda x: x[1], ascending = False)
grouped.take(10)

24/03/11 13:04:37 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/03/11 13:04:37 ERROR Utils: Uncaught exception in thread stop-spark-context
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.deploy.client.StandaloneAppClient.stop(StandaloneAppClient.scala:288)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.org$apache$spark$scheduler$cluster$StandaloneSchedulerBackend$$stop(StandaloneSchedulerBackend.scala:275)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.stop(StandaloneSchedulerBackend.scala:142)
	at org.apache.spark.scheduler.SchedulerBackend.stop(SchedulerBackend.scala:33)
	at org.apache.spark.scheduler.SchedulerBackend.

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Master removed our application: KILLED
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1247)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adapted(DAGScheduler

In [35]:
grouped = grouped.sortBy(lambda x: x[1], ascending = True)
grouped.take(10)

[('AskReddit', (-121714, 589947)),
 ('relationships', (-75276, 352049)),
 ('tifu', (-17853, 52219)),
 ('relationship_advice', (-7130, 50416)),
 ('funny', (-6758, 40171)),
 ('WTF', (-6555, 25781)),
 ('trees', (-6271, 47286)),
 ('AdviceAnimals', (-6257, 40783)),
 ('politics', (-6144, 36518)),
 ('offmychest', (-5298, 17175))]

In [38]:
grouped = grouped.filter(lambda x: x[1][1] > 50)
grouped = grouped.sortBy(lambda x: x[1][1], ascending = True)
grouped.take(10)

[('LiverpoolFC', (13, 51)),
 ('Surface', (11, 51)),
 ('gamecollecting', (3, 51)),
 ('law', (1, 51)),
 ('italy', (0, 51)),
 ('4chan', (-7, 51)),
 ('snowboarding', (14, 52)),
 ('aspergers', (6, 52)),
 ('martialarts', (5, 52)),
 ('Buddhism', (2, 52))]